In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
from dash import Dash, html, dash_table, dcc, Input, Output
import plotly.express as px

In [2]:
# Upload data
df = "Clean_Australian_Shark_Attacks.csv"
df

clean_df = pd.read_csv(df, encoding = "CP1252")
clean_df

,Unnamed: 0,Incident.year,Victim.injury,State,Latitude,Longitude,Shark.common.name,Shark.scientific.name,Provoked/unprovoked,Site.category,Incident.month
0,0,1791,fatal,NSW,-33.86666667,151.2,white shark,Carcharodon carchariasÂ,unprovoked,coastal,1
1,1,1803,injured,WA,-25.83333333,113.8833333,tiger shark,Galeocerdo cuvierÂ,unprovoked,coastal,3
2,2,1807,injured,NSW,-33.86666667,151.2,bull shark,Carcharhinus leucas,unprovoked,estuary/harbour,1
3,3,1820,fatal,TAS,-42.8,147.5333333,unknown,unknown,provoked,coastal,1
4,4,1825,injured,NSW,-33.85,151.2166667,bull shark,Carcharhinus leucas,unprovoked,estuary/harbour,1
...,...,...,...,...,...,...,...,...,...,...,...
1223,1223,2023,injured,QLD,-14.06549,144.26335,unknown,unknown,provoked,coastal,11
1224,1224,2023,injured,WA,-30.81556,115.19234,bronze whaler shark,Carcharhinus brachyurus,unprovoked,island open ocean,12
1225,1225,2023,injured,NSW,-31.9694,152.5863946,unknown,unknown,unprovoked,coastal,12
1226,1226,2023,fatal,SA,-34.679356,137.684906.,white shark,Carcharodon carcharias,unprovoked,coastal,12


In [3]:
# Checking the number of records.
# total_records = len(clean_df['Unnamed: 0'].unique())
# total_records

In [4]:
# select my chosen columns focusing on shark types based on state
reduced_df = clean_df.loc[:, ["Shark.common.name", "State"]]
reduced_df

,Shark.common.name,State
0,white shark,NSW
1,tiger shark,WA
2,bull shark,NSW
3,unknown,TAS
4,bull shark,NSW
...,...,...
1223,unknown,QLD
1224,bronze whaler shark,WA
1225,unknown,NSW
1226,white shark,SA


In [5]:
# an overall total count by shark common name
total_name = reduced_df['Shark.common.name'].value_counts()
total_name

Shark.common.name
white shark                  372
tiger shark                  229
wobbegong                    203
bull shark                   201
whaler shark                  75
unknown                       61
bronze whaler shark           29
grey nurse shark               9
grey reef shark                8
whitetip reef shark            7
lemon shark                    6
dusky shark                    5
hammerhead shark               5
blacktip reef shark            4
galapagos shark                3
shortfin mako shark            2
broadnose sevengill shark      2
dogfish                        1
school shark                   1
sevengill shark                1
silvertip shark                1
blind shark                    1
port jackson shark             1
seven gill shark               1
Name: count, dtype: int64

In [8]:
# count by shark name for each state
totals_df = reduced_df[['State', 'Shark.common.name']].value_counts().reset_index(name='count')
totals_df.head(20)

,State,Shark.common.name,count
0,QLD,tiger shark,179
1,NSW,white shark,155
2,NSW,wobbegong,107
3,NSW,bull shark,101
4,QLD,bull shark,79
5,WA,white shark,75
6,SA,white shark,63
7,WA,wobbegong,52
8,VIC,white shark,46
9,QLD,whaler shark,28


In [9]:
totals_df.groupby(['State']).value_counts().head(20)

State  Shark.common.name    count
NSW    blind shark          1        1
       bronze whaler shark  11       1
       bull shark           101      1
       dogfish              1        1
       dusky shark          4        1
       galapagos shark      3        1
       grey nurse shark     7        1
       tiger shark          15       1
       unknown              15       1
       whaler shark         26       1
       white shark          155      1
       wobbegong            107      1
NT     whaler shark         4        1
       bull shark           3        1
       tiger shark          9        1
       unknown              1        1
       whitetip reef shark  1        1
       wobbegong            1        1
QLD    wobbegong            22       1
       blacktip reef shark  2        1
Name: count, dtype: int64

In [10]:
dff = totals_df
dff

,State,Shark.common.name,count
0,QLD,tiger shark,179
1,NSW,white shark,155
2,NSW,wobbegong,107
3,NSW,bull shark,101
4,QLD,bull shark,79
...,...,...,...
58,NT,wobbegong,1
59,NT,whitetip reef shark,1
60,NT,unknown,1
61,NSW,dogfish,1


In [11]:
# Get unique states and shark common names for dropdown options
states = dff['State'].unique()
names = dff['Shark.common.name'].unique()

In [12]:
# Initialize the app
app = Dash(__name__)

NameError: name 'Dash' is not defined

In [13]:
app.layout = html.Div([
    html.H1("Shark Attack Counts by Common Name and State"),
# Dropdown menu for selecting states
    dcc.Dropdown(
        id='state-dropdown',
        options=[{'label': state, 'value': state} for state in states],
        value='NSW',  # Default value
        multi=False,  # Allow single selection
        style={'width': '50%'}
    ),
    # Graph component for displaying the bar chart
    dcc.Graph(id='bar-chart')
])
    
# Define callback to update the bar chart based on dropdown selection
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('state-dropdown', 'value')]
)

def update_bar_chart(selected_state):
    filtered_dff = dff[dff['State'] == selected_state]
    fig = px.bar(filtered_dff, x='Shark.common.name', y='count', color='Shark.common.name',
                 labels={'count': 'Number of Attacks', 'Shark.common.name': 'Shark Common Name'})
    fig.update_layout(title=f'Shark Attack Counts by name in {selected_state}')
    return fig
    
    
        
# run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=2226)

OSError: Address 'http://127.0.0.1:2224' already in use.
    Try passing a different port to run_server.